# Flan T5 Summarization

https://github.com/HarrywillDr/KeyPoint-Analysis/blob/main/KPG/kpa_t5.py

## Import packages

In [1]:
import pandas as pd
from argsum import load_kp_test, preprocess_input_t5, train_t5, get_t5_cluster_sums

/Users/moritz/miniconda3/envs/AM/lib/python3.8/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/Users/moritz/miniconda3/envs/AM/lib/python3.8/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
loading configuration file config.json from cache at /Users/moritz/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/6cdc0aad91f5ae2e6712e91bc7b65d1cf5c05411/config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMa

## Load data

In [2]:
# Load ArgKP21
df = pd.read_csv('data/ArgKP-2021/dataset_splits.csv')
# Load ArgKP21 train/dev splits
df_train = df[df['set'] == 'train']
df_dev = df[df['set'] == 'dev']
# Load KP_test
KP_test = load_kp_test()
# Preprocess ArgKP21 train/dev splits
df_train = preprocess_input_t5(df_train)
df_dev = preprocess_input_t5(df_dev)

In [22]:
train_t5(df_train, df_dev, learning_rate = 4e-4, load_best_model_at_end = False)

{'loss': 9.3621, 'learning_rate': 0.000358974358974359, 'epoch': 1.54}
{'eval_loss': 0.5509318113327026, 'eval_runtime': 1.1974, 'eval_samples_per_second': 30.064, 'eval_steps_per_second': 2.505, 'epoch': 1.54}
{'loss': 0.3898, 'learning_rate': 0.0003179487179487179, 'epoch': 3.08}
{'eval_loss': 0.14726188778877258, 'eval_runtime': 1.1434, 'eval_samples_per_second': 31.486, 'eval_steps_per_second': 2.624, 'epoch': 3.08}
{'loss': 0.1226, 'learning_rate': 0.00027692307692307695, 'epoch': 4.62}
{'eval_loss': 0.1561894416809082, 'eval_runtime': 1.0047, 'eval_samples_per_second': 35.833, 'eval_steps_per_second': 2.986, 'epoch': 4.62}
{'loss': 0.0737, 'learning_rate': 0.00023589743589743593, 'epoch': 6.15}
{'eval_loss': 0.17279836535453796, 'eval_runtime': 1.0187, 'eval_samples_per_second': 35.337, 'eval_steps_per_second': 2.945, 'epoch': 6.15}
{'loss': 0.043, 'learning_rate': 0.00019487179487179487, 'epoch': 7.69}
{'eval_loss': 0.20564384758472443, 'eval_runtime': 1.0222, 'eval_samples_per_

## Inspect summaries of fine-tuned T5

In [27]:
topic = KP_test['topic'].unique()[0]
stance = KP_test['stance'].unique()[1]
kps = KP_test[(KP_test['topic'] == topic) & (KP_test['stance'] == stance)]['key_point'].to_list()
kps_unique = set(list(kps))
args = KP_test[(KP_test['topic'] == topic) & (KP_test['stance'] == stance)]['argument'].to_list()
kps_unique_dict = dict(zip(kps_unique, [i for i in range(len(kps_unique))]))
gold_cluster_ids = [kps_unique_dict[kps[i]] for i in range(len(kps))]

In [28]:
kps_unique

{'Mandatory vaccination contradicts basic rights',
 'Routine child vaccinations are not necessary to keep children healthy',
 'Routine child vaccinations, or their side effects, are dangerous',
 'The parents and not the state should decide'}

In [29]:
sums = get_t5_cluster_sums(args, gold_cluster_ids, topic, stance, num_beams = 6, n = 5)
sums

Get Flan-T5 Cluster Sums:   0%|          | 0/1 [00:00<?, ?it/s]

{0: ['Child vaccination should be mandatory because they cannot catch the virus',
  'Child vaccination should be mandatory since they do not have symptoms',
  'Child vaccination should not be mandatory because they cannot catch the virus',
  'Child vaccination should not be mandatory since they cannot catch the virus',
  'Child vaccination should be mandatory as they do not have symptoms'],
 1: ['Parents should be permitted to choose whether or not to vaccinate their children',
  'Parents should be permitted to choose whether or not to vaccinate their child',
  'Parents should be permitted to choose whether to vaccinate their children',
  'Parents should be permitted to choose the vaccinations of their children',
  'Parents should choose for their children'],
 2: ['Vaccination is not mandatory because everyone should choose for themselves whether to be vaccinated',
  'It is not mandatory for everyone to be vaccinated',
  'It is within the freedom of the people to choose for themselves 